#### Chrysanthos Lianos
#### MSc Data Science 2016 Part Time
#### Machine Learning and Computational statistics Project

### First Part

In [13]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image as image
import pandas as pd
import scipy 
%matplotlib inline

In [54]:
def getdf(type):
    tables = []
    for i in range(10):
        temp_df = pd.read_csv('D:/mnistdata/{}{}.txt'.format(str(type),i),sep = ' ', names = range(784))
        temp_df['target'] = i
        tables.append(temp_df)
    tables = pd.concat(tables)
    return tables

#testset,trainset = read_data()

Now get the data
train_X is the set of training data, test_X is test
train_Y is the OneHot column for train_X and also has column which is aggregation of values eg 0: 100000000

In [84]:
train_X = getdf('train')
test_X = getdf('test')
#yyy = oneHotIt(train_X)
# create train_y 
enc = OneHotEncoder(dtype='int') # create int type to concatenate later
train_Y = np.asmatrix(enc.fit_transform(train_X['target'].reshape(-1,1)).toarray())
test_Y =  np.asmatrix(enc.fit_transform(test_X['target'].reshape(-1,1)).toarray())
#train_Y = pd.DataFrame(enc.fit_transform(train_X['target'].reshape(-1,1)).toarray()) #training for Y
#train_Y['aggr'] = train_Y.apply(lambda row: ''.join(map(str, row)), axis=1)
# in order to access eg the first row of the train_X use .iloc[0]
# to see all rows whose target is in range (0..9) use train_X.loc[trainX['target'] == 0]
#train_X.loc[train_X['target']== 0].sample(5)
#len(train_X.loc[train_X['target']== 1])
# X is train_X.ix[:,:784] and Y is train_X['target']
X = train_X.ix[:,:784]
# add 1 infront of X
X.insert(0,'ones',1)
t = train_X['target']
N,D = X.shape
#set params
l = 0.5
winit = np.zeros(D)
options = [500,1/1000000 ,8/N]  # accuracy of 6 decimal places

In [85]:
def sigmoid(z):
    yx = 1.0/(1+np.exp(-z))
    return yx

def ml_logregTrain(t,X,l,winit,options):
    w = winit
    iterations = options[0]
    #tol=1/10
    tol = options[1]
    eta = options[2]
    ewold = float('-inf')
    for it in range(0,iterations):
        yx = np.dot(X,w)
        s = sigmoid(yx)
        ew = sum(np.matmul(t.T,s) - np.log(sum(np.exp(s))))
        #ew = sum(np.matmul(t,s)) - np.log(sum(np.exp(s)))
        #ew = sum(np.multiply(t,np.log(s)) + np.multiply((1-t),np.log(1-s))) - np.dot((0.5 * l), np.dot(w.transpose(),w))  
        print('Iteration: {}, Cost function: {}'.format(it,ew,ewold))
        if np.absolute(ew -ewold) < tol:
            break
        gradient = np.dot((t-s).transpose(),X) - np.multiply(l,w)
        #gradient = np.dot(X.transpose(),(t-s)) - np.multiply(l,w)
        w += np.multiply(np.float(eta),gradient)
        ewold=ew
    ew =0
    return w 


#w = ml_logregTrain(train_Y['aggr'], X, l, winit, options)
#w = ml_logregTrain(train_X['target'],X,l,winit,options)
w = ml_logregTrain(train_Y,X,l,winit,options)
#w = ml_logregTrain(train_Y,X,l,winit,options)

Iteration: 0, Cost function: [[ 2949.99790016  3359.49790016  2967.49790016  3053.99790016
   2909.49790016  2698.99790016  2947.49790016  3120.99790016
   2913.99790016  2962.99790016]]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()